In [3]:
import re
import html
from collections import Counter
import numpy as np
import os
import pickle
import collections
import sys
import pandas as pd
import sklearn
from pathlib import Path
import nltk
import tokenize
np.random.seed(1234)

C:\Users\Remi\Miniconda3\envs\fastai\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


# Wikitext part 

In [63]:
save_path = r"F:\data\text_classification\preprocessed_wiki\\"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [87]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [88]:
corpus_train  = []
corpus_test   = []

with open("F:\data\wikitext-103-v1\wikitext-103\wiki.train.tokens", "r", encoding="utf8") as f:
    for i in f:
        if i.find("=")==-1 and len(i) > 15: #not a \n line of a supersmall line 
            corpus_train.append(fixup(i).split(" "))         
with open("F:\data\wikitext-103-v1\wikitext-103\wiki.test.tokens", "r", encoding="utf8") as f:
    for i in f:
        if i.find("=")==-1 and len(i) > 15: #not a \n line a supersmall line 
            corpus_test.append(fixup(i).split(" "))

In [89]:
print(len(corpus_train))
len((corpus_val))

846176


2091

In [90]:
vocab_size = 30000
freq       = Counter(p for o in corpus_train+corpus_val for p in o)
freq.most_common(vocab_size)

[('the', 5580633),
 (',', 4969042),
 ('.', 3628837),
 ('of', 2723158),
 ('and', 2458189),
 ('to', 1972724),
 ('in', 1882952),
 ('a', 1675502),
 ('"', 1337140),
 ('was', 1077209),
 ('The', 849891),
 ('', 848267),
 ('\n', 848267),
 ("'s", 725981),
 ('on', 683132),
 ('that', 680199),
 ('as', 674890),
 ('for', 674140),
 ('with', 631106),
 ('by', 598734),
 (')', 571620),
 ('(', 571313),
 ('is', 511953),
 ('his', 437234),
 ('from', 431818),
 ('u_n', 420222),
 ('at', 415870),
 ('were', 355455),
 ('it', 326585),
 ('he', 318997),
 ('an', 306099),
 ('had', 293131),
 ('In', 279903),
 ('which', 275809),
 ('be', 234475),
 ('but', 203545),
 ('are', 203142),
 ('not', 194728),
 (';', 190579),
 ('their', 186708),
 ('first', 182485),
 ('also', 179984),
 ('her', 176682),
 ('–', 176595),
 (':', 175614),
 ('its', 174974),
 ('or', 166096),
 ('have', 163545),
 ('who', 155628),
 ('one', 152974),
 ('been', 152313),
 ('has', 150710),
 ('@,@', 149273),
 ('this', 145568),
 ('two', 144307),
 ('He', 141449),
 ("'",

In [91]:
string_list = [ s for s,i in freq.most_common(vocab_size) if i>2 ]
string_list.insert(0,"_unk_")
string_list.insert(1,"_pad_")

In [92]:
len(string_list)

30002

In [93]:
itos        = {st:index for st, index in enumerate(string_list)}
stoi        = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(string_list)})

In [94]:
string_list[2500]

'hundred'

In [95]:
total_len_train = sum([len(a) for a in corpus_train])

In [96]:
tokken_corpus_train = []
tokken_corpus_val   = []

for i in range(len(corpus_train)):
    for a in corpus_train[i]:
        tokken_corpus_train.append(stoi[a])
        
for i in range(len(corpus_val)):
    for a in corpus_val[i]:
        tokken_corpus_val.append(stoi[a])

In [97]:
len(tokken_corpus_train) == total_len_train

True

In [98]:
np.save(os.path.join(save_path + 'array_wikitext_train.npy'),np.array(tokken_corpus_train)[:-250000])
np.save(os.path.join(save_path + 'array_wikitext_val.npy') , np.array(tokken_corpus_train)[-250000:])
np.save(os.path.join(save_path + 'array_wikitext_test.npy'), np.array(tokken_corpus_val))

In [99]:
with open(os.path.join(save_path + 'string_list_wiki.pkl'), 'wb') as f:
   pickle.dump(string_list, f, pickle.HIGHEST_PROTOCOL)
# with open(r"F:\data\wikitext-103-v1\wikitext-103\preprocessed\itos.pkl", 'wb') as f:
#    pickle.dump(itos, f, pickle.HIGHEST_PROTOCOL)

# IMDB part 

### We first load all our files in memory (trn_texts,trn_labels)

In [4]:
working_folder_path= r'F:\data\aclImdb1'

In [5]:
class_path = os.path.join(working_folder_path, r'imdb_clas')
if not os.path.exists(class_path):
    os.makedirs(class_path)

lm_path = os.path.join(working_folder_path, r'imdb_lm')
if not os.path.exists(lm_path):
    os.makedirs(lm_path)

In [208]:
CLASSES = ['neg', 'pos', 'unsup']

def get_texts(path):
    texts,labels = [],[]
    for idx,label in enumerate(CLASSES):
        print(os.path.join(working_folder_path, path,label))
        for fname in Path(os.path.join(working_folder_path, path,label)).glob('*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(idx)
    return np.array(texts),np.array(labels)

trn_texts,trn_labels = get_texts('train')
val_texts,val_labels = get_texts('test')

F:\data\aclImdb1\train\neg
F:\data\aclImdb1\train\pos
F:\data\aclImdb1\train\unsup
F:\data\aclImdb1\test\neg
F:\data\aclImdb1\test\pos
F:\data\aclImdb1\test\unsup


Then we shuffle it and save it

In [209]:
trn_idx    = np.random.permutation(len(trn_texts))
val_idx    = np.random.permutation(len(val_texts))

trn_texts  = trn_texts[trn_idx]
val_texts  = val_texts[val_idx]

trn_labels = trn_labels[trn_idx]
val_labels = val_labels[val_idx]

In [211]:
col_names = ['labels','text']
df_trn    = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=col_names)
df_val    = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=col_names)

In [212]:
df_trn.to_csv(os.path.join(class_path,'train.csv'),header=False, index=False)
df_val.to_csv(os.path.join(class_path,'test.csv'), header=False, index=False)

In [216]:
class_path

'F:\\data\\aclImdb1\\imdb_clas'

In [213]:
import gc
gc.collect()

57

In [215]:
trn_labels

array([2, 0, 1, ..., 2, 0, 1])

In [47]:
# concat = np.concatenate([trn_texts,val_texts], axis=0)

In [61]:
# concat.shape
# del concat

In [ ]:
# from sklearn.model_selection import train_test_split

# trn_texts,val_texts = train_test_split( concat[:50000], test_size=0.1)

# df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=col_names)
# df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=col_names)

# df_trn.to_csv(os.path.join(lm_path,'train.csv'), header=False, index=False)
# df_val.to_csv(os.path.join(lm_path,'test.csv') , header=False, index=False)

### Tokenisation part: 

In [1]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

In [6]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [219]:
def get_texts(df):
    texts = f'\n{BOS} {FLD} 1 ' + df.iloc[:,1].astype(str)
    texts = list(texts.apply(fixup).values)
    dfREMI = pd.DataFrame(texts)[0].apply(nltk.word_tokenize)

    return pd.concat([df, dfREMI], axis = 1)

In [221]:
df_trn = pd.read_csv(os.path.join(class_path,'train.csv'), header=None)
df_val = pd.read_csv(os.path.join(class_path,'test.csv'), header=None)

In [222]:
trn_df = get_texts(df_trn)
val_df = get_texts(df_val)

In [223]:
trn_df.to_csv(os.path.join(class_path,'train_tokenized.csv'), header=False, index=False)
val_df.to_csv(os.path.join(class_path,'val_tokenized.csv')  , header=False, index=False)

### New vocab 60k

We will then load our data in order to convert each word to an index. We will use each of the 3 classes in order to have a sufficient vocabulary size.

In [27]:
trn_df = pd.read_csv(os.path.join(class_path,'train_tokenized.csv'), header=None)
val_df = pd.read_csv(os.path.join(class_path,'val_tokenized.csv'),  header=None)

In [28]:
freq = Counter(p for o in trn_df.iloc[:,2].tolist() for p in o[2:-2].split("\', \'"))

In [42]:
stringlist_imdb = [s for s,i in freq.most_common(60000) if i >2]

In [43]:
len(itos_imdb)

60002

In [44]:
stringlist_imdb.insert(0, '_pad_')
stringlist_imdb.insert(0, '_unk_')

In [45]:
stringlist_imdb[0:10]

['_unk_', '_pad_', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

We can therefore create our string to integer dictonary (stoi_imdb)

In [68]:
stoi_imdb = collections.defaultdict(lambda:0,  {s:i for i,s in enumerate(stringlist_imdb)})

In [47]:
#all our training sentences in on array, used for the second pretraining phase.

arr_train = [stoi_imdb[p] for o in trn_df[2].tolist() for p in o[2:-2].split("\', \'")]
np.save(r"F:\data\aclImdb1\pretrain\array_imdb_train.npy", np.array(arr_train) )

In [48]:
len(arr_train)

19331955

In [49]:
input_train = [ [stoi_imdb[p] for p in o[2:-2].split("\', \'")] for o in trn_df[trn_df[0]!=2][2].tolist()]
input_test  = [ [stoi_imdb[p] for p in o[2:-2].split("\', \'")] for o in val_df[2].tolist()]

In [50]:
y_train = trn_df[trn_df[0]!=2][0].values.transpose()
y_test  = val_df[0].values.transpose()

In [51]:
np.save(r"F:\data\aclImdb1\pretrain\input_train.npy", np.array(input_train) )
np.save(r"F:\data\aclImdb1\pretrain\input_test.npy", np.array(input_test ) )

In [52]:
np.save(r"F:\data\aclImdb1\pretrain\y_train.npy"    , np.array(y_train) )
np.save(r"F:\data\aclImdb1\pretrain\y_test.npy"    , np.array(y_test ) )

In [54]:
# with open(r"F:\data\aclImdb1\pretrain\stoi_idbm.pkl", 'wb') as f:
#    pickle.dump(stoi_imdb, f, pickle.HIGHEST_PROTOCOL)
with open(r"F:\data\text_classification\preprocessed_imdb\stringlist_imdb.pkl", 'wb') as f:
   pickle.dump(stringlist_imdb, f, pickle.HIGHEST_PROTOCOL) 

### Matrix embedding

In [56]:
import torch

In [57]:
PATH_model            = r'F:\data\text_classification\model_sav\model_wiki103_10-3.pth'

In [58]:
encoder_weight_array  = np.array(torch.load(PATH_model)['0.encoder.weight'])

In [59]:
mean_vector           = encoder_weight_array.mean(0)

In [60]:
mean_vector.shape

(400,)

In [65]:
with open(r"F:\data\text_classification\preprocessed_wiki\string_list_wiki.pkl", 'rb') as f:
    string_list_wiki = pickle.load(f)

In [66]:
new_weight = np.zeros((len(itos_imdb),400))

In [67]:
new_weight.shape

(60002, 400)

In [73]:
len(stringlist_imdb), len(string_list_wiki)

(60002, 30002)

In [76]:
stoi_wikitext = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(string_list_wiki)})

In [54]:
# stoi_wikitext  = collections.defaultdict(lambda:0 , {s:i for i,s in itos_wikitext.items()})

In [77]:
for index_imdb ,string_imdb in enumerate(stringlist_imdb):
    indice                 = stoi_wikitext[string_imdb]
    new_weight[index_imdb] = encoder_weight_array[indice] if indice > 0 else  mean_vector

We do a last checking before we save our new matrix

In [87]:
string_list_wiki[2500]

'hundred'

In [85]:
new_weight[3149][0:10]

array([ 0.25644088, -0.31902361, -0.0967235 , -0.38573858, -0.64922625,
        1.0714463 ,  0.27908069,  0.12309778, -0.00300951,  0.51692551])

In [82]:
stringlist_imdb.index("hundred")

3149

In [83]:
encoder_weight_array[2500][0:10]

array([ 0.25644088, -0.31902361, -0.0967235 , -0.38573858, -0.64922625,
        1.0714463 ,  0.27908069,  0.12309778, -0.00300951,  0.51692551], dtype=float32)

In [89]:
np.save(r'F:\data\text_classification\model_sav\imdb_embedding_60002.npy',new_weight)

### debuging part 

In [58]:
stoi_wikitext["table"]

2500

In [76]:
stoi = {s:i for i,s in enumerate(itos_imdb)}
stoi_imdb["table"]

2601

In [62]:
encoder_weight_array[2500, 0:10]

array([ 1.17675555,  1.19893169, -1.13339305, -1.79974651,  1.11936235,
       -0.58095008,  0.39032462, -1.65573072, -3.34352851, -0.79369217], dtype=float32)

In [63]:
new_weight[2601, 0:10]

array([ 1.17675555,  1.19893169, -1.13339305, -1.79974651,  1.11936235,
       -0.58095008,  0.39032462, -1.65573072, -3.34352851, -0.79369217])

In [90]:
i = 0
index_imdb_list = []
for index_imdb ,string_imdb in enumerate(itos_imdb):
    indice                 = stoi_wikitext[string_imdb]
    i = i+1 if indice > 0 else i
    if indice > 0:
            index_imdb_list.append(index_imdb)


In [95]:
index_imdb_list[-100]

58914

In [88]:
print(f"overlaping coef: {i/30000}")

overlaping coef: 0.7398333333333333
